# Stundenplanungsproblem - Constraint Programmming

Alternativ zu der gemischt ganzzahligen Programmierung, können Stundeplanungsprobleme (sowie viele andere Planungs- und Optimierungsprobleme) auch mit Constraint Programming gelöst werden.

Wir verwenden hier den kostenlos zugänglichen SAT-Solver von Google OR-Tools, aber es gibt diverse andere Solver (z.B. CP Optimizer von IBM), welche ebenfalls verwendet werden können. Anders als MIP Solver haben CP Solver viele Eigenheiten. Nicht alle Konzepte die in OR-Tools funktionieren können nahtlos in CP Optimizer verwendet werden (und natürlich auch umgekehrt). Es gibt zwar Versuche, eine gemeinsame Modelliersprache zu erschaffen (z.B. MiniZinc), jedoch gibt es kein gemeinsames Format wie es mit dem LP oder MPS-Format für MIP Solver normal ist.

Auf der offiziellen Website von OR-Tools ist eine sehr gute Dokumentation vorhanden: https://developers.google.com/optimization/reference/python/index_python

Wir importieren zuerst den OR-Tools SAT-Solver:

In [1]:
from ortools.sat.python import cp_model

Wir verwenden widerum pandas um die Daten einzulesen:

In [2]:
import pandas as pd
df = pd.read_csv('data.csv')
df.head()

,ID,Fach,Lehrperson,Stunden,Klasse,Overlap
0,1,SH,bip,2,U21a,23
1,2,NT,arj,2,U21a,0
2,3,RE,blb,2,U21a,0
3,4,DE,scn,2,U21a,0
4,5,TG,gej,2,U21a,0


Wie schon im MIP Modell, werden wir zuerst die Daten in einfach zugänglichen Datenstrukturen zugänglich machen:

In [3]:
lectures = list()
classes = set()
teachers = set()
subjects = set()
lectures_by_class = dict()
lectures_by_teacher = dict()
lecture_details = dict()

for index, row in df.iterrows():
    lectures.append(row['ID'])
    lecture_details[row['ID']] = {'subject': row['Fach'], 'teacher': row['Lehrperson'], 'hours': row['Stunden'], 'class': row['Klasse'], 'overlap': row['Overlap']}
    classes.add(row['Klasse'])
    teachers.add(row['Lehrperson'])
    subjects.add(row['Fach'])
    if row['Klasse'] in lectures_by_class:
        lectures_by_class[row['Klasse']].append(row['ID'])
    else:
        lectures_by_class[row['Klasse']] = [row['ID']]
    if row['Lehrperson'] in lectures_by_teacher:
        lectures_by_teacher[row['Lehrperson']].append(row['ID'])
    else:
        lectures_by_teacher[row['Lehrperson']] = [row['ID']]

Wir definieren nun ebenfalls unsere Tage und Zeitslots. Zusätzlich werden wir basierend darauf auch noch fortlaufende Zeitslots definieren, welche wir für die Modellierung brauchen werden.

In [4]:
days = [0,1,2,3,4] # Mo, Di, Mi...
timeslots = [0,1,2,3,4,5,6,7,8,9,10] # 08:00 - 08:45, 08:50 - 09:35, 09:55 - 10:40...
slots = [d*len(timeslots)+ts for d in days for ts in timeslots] # Example: slot 11 is timeslot 0 on day 1

Nun kann das CP Modell deklariert werden und die Variablenstruktur festgelegt werden.
Im MIP Modell wurden binäre Variablen verwendet, um für jeden Tag und Zeitslot zu entscheiden, ob eine Lektion stattfindet oder nicht. Hier verwenden wir einen total anderen Anlass:

Jede Lektion hat eine ganzzahlige Start und Endvariable. Da wir die Dauer jeder Lektion kennen, ist die Endvariable einfach zu berechnen. Wir können zusätzlich auch eine Intervallvariable einführen, welche es später leichter macht, gewisse Sachen auszudrücken:

In [5]:
model = cp_model.CpModel()
start_vars = dict()
end_vars = dict()
durations = dict()
interval_vars = dict()
max_time = len(slots)

# Create variables
for lecture in lectures:
    start_vars[lecture] = model.NewIntVar(0, max_time, 'start' + str(lecture))
    end_vars[lecture] = model.NewIntVar(0, max_time, 'end' + str(lecture))
    durations[lecture] = lecture_details[lecture]['hours']
    interval_vars[lecture] = model.NewIntervalVar(start_vars[lecture], durations[lecture], end_vars[lecture], 'interval' + str(lecture))
    

Sowohl für die Lehrer, als auch für die Schüler soll es keine Überlappungen von Lektionen geben. 
Dank der Interval-Variablen, welche wir im vorherigen Schritt definiert haben, kann dies sehr leicht ausgedrückt werden:

In [6]:
for c in classes:
    model.AddNoOverlap([interval_vars[l] for l in lectures_by_class[c]])
                        
for t in teachers:
    model.AddNoOverlap([interval_vars[l] for l in lectures_by_teacher[t] if lecture_details[l] == 0])

Wie schon im MIP-Modell möchten wir sicherstellen, dass die Sportlektionen der beiden Klassen gleichzeitig stattfinden.
Auch hier hätte es wohl Sinn gemacht, statt zwei Anlässe, nur einen zu definieren.

In [7]:
for l in lectures:
    overlap = lecture_details[l]['overlap']
    if overlap > 0:
        model.Add(start_vars[l] == start_vars[overlap])

Damit haben wir die Grundstruktur und die wichtigsten Regeln festgelegt und können nun grundsätzlich einen 
Stundenplan generieren, indem wir den Solver (hier den SAT-Solver von Google OR-Tools) aufrufen:

In [8]:
cb = cp_model.ObjectiveSolutionPrinter()
solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 30 # We just set some random time limit
solver.parameters.num_search_workers = 4
status = solver.SolveWithSolutionCallback(model, cb)

Solution 0, time = 0.01 s, objective = 0
Solution 1, time = 0.01 s, objective = 0
Solution 2, time = 0.01 s, objective = 0
Solution 3, time = 0.01 s, objective = 0


Wir definieren widerum eine Funktion um die Lösung auf eine einfache Art und Weise zu visualisieren:

In [9]:
def visualize():
    for c in classes:
        df = pd.DataFrame(columns=days, index=timeslots)
        for d in days:
            for s in timeslots:
                empty = True
                for l in lectures_by_class[c]:
                    if solver.Value(start_vars[l]) <= d*11+s and solver.Value(end_vars[l]) >= d*11+s+1:
                        df.at[s,d] = lecture_details[l]['subject'] + ' / ' + lecture_details[l]['teacher']
                        empty = False
                if empty:
                    df.at[s,d] = ''
        from IPython.display import display, HTML
        print(c)
        display(HTML(df.to_html()))

In [10]:
visualize()

U21b


,0,1,2,3,4
0,MA / buk,TG / gyg,MA / buk,NT / arj,
1,BL / frm,GS / dua,MA / buk,DE / frm,
2,IN / frc,GS / dua,FR / jeg,DE / frm,
3,KS / frm,DE / frm,FR / jeg,,
4,DE / frm,DE / frm,MU / hlu,,
5,SH / bip,EN / fsl,MU / hlu,,
6,FR / jeg,EN / fsl,RE / blb,,
7,EN / fsl,GG / buk,RE / blb,,
8,SH / bip,GG / buk,MA / buk,,
9,SH / bip,BG / gyg,MA / buk,,


U21a


,0,1,2,3,4
0,KS / bip,NT / arj,MU / vom,MA / ale,
1,BL / bip,RE / blb,MU / vom,BG / gej,
2,MA / ale,RE / blb,DE / scn,BG / gej,
3,IN / frc,DE / scn,DE / scn,,
4,DE / scn,DE / scn,GS / dua,,
5,SH / bip,TG / gej,GS / dua,,
6,EN / dea,TG / gej,MA / ale,,
7,FR / ebm,FR / ebm,MA / ale,,
8,SH / bip,FR / ebm,GG / bip,,
9,SH / bip,EN / dea,GG / bip,,


Wie schon mit dem MIP haben wir auch hier einen Stundenplan erhalten. 
Wir fügen nun die selben zusätzlichen Bedingungen ein.

Zuerst werden wir sicherstellen, dass es keine Lektionen am Mittwochnachmittag gibt. Wir könnten dazu grundsätzlich alle Startzeiten für Lektionen am Mittwochnachmittag verbieten. Stattdessen werden wir aber einen anderen Trick anwenden, und eine weitere Intervallvariable für den Mittwochnachmittag einfügen (mit gegebenen Start- und Endzeiten) und dann mithilfe der NoOverlap-Bedingungen sicherstellen, dass dieses Interval für keine der Klassen doppelt belegt ist:

In [11]:
mittwoch_nachmittag = model.NewIntervalVar(27, 6, 33, 'mittwoch_nachmittag')

for c in classes:
    model.AddNoOverlap([interval_vars[l] for l in lectures_by_class[c]]+[mittwoch_nachmittag])

In [12]:
status = solver.SolveWithSolutionCallback(model, cb)

Solution 4, time = 0.06 s, objective = 0
Solution 5, time = 0.06 s, objective = 0
Solution 6, time = 0.06 s, objective = 0
Solution 7, time = 0.06 s, objective = 0


In [13]:
visualize()

U21b


,0,1,2,3,4
0,MA / buk,TG / gyg,MA / buk,MU / hlu,
1,BL / frm,GS / dua,MA / buk,MU / hlu,
2,IN / frc,GS / dua,FR / jeg,RE / blb,
3,KS / frm,DE / frm,FR / jeg,RE / blb,
4,DE / frm,DE / frm,,MA / buk,
5,SH / bip,EN / fsl,,MA / buk,
6,FR / jeg,EN / fsl,,NT / arj,
7,EN / fsl,GG / buk,,NT / arj,
8,SH / bip,GG / buk,,DE / frm,
9,SH / bip,BG / gyg,,DE / frm,


U21a


,0,1,2,3,4
0,KS / bip,NT / arj,MU / vom,GS / dua,
1,BL / bip,RE / blb,MU / vom,GS / dua,
2,MA / ale,RE / blb,DE / scn,MA / ale,
3,IN / frc,DE / scn,DE / scn,MA / ale,
4,DE / scn,DE / scn,,GG / bip,
5,SH / bip,TG / gej,,GG / bip,
6,EN / dea,TG / gej,,MA / ale,
7,FR / ebm,FR / ebm,,MA / ale,
8,SH / bip,FR / ebm,,BG / gej,
9,SH / bip,EN / dea,,BG / gej,


In [14]:
# Slot 5 soll grundsätzlich als Mittagspause frei bleiben:

for l in lectures:
    for d in days:
        model.Add(start_vars[l] != 5+d*11)
        if lecture_details[l]['hours'] == 2:
            model.Add(start_vars[l] != 4+d*11)

In [15]:
status = solver.SolveWithSolutionCallback(model, cb)
visualize()

Solution 8, time = 0.13 s, objective = 0
Solution 9, time = 0.13 s, objective = 0
Solution 10, time = 0.13 s, objective = 0
Solution 11, time = 0.13 s, objective = 0
U21b


,0,1,2,3,4
0,MA / buk,TG / gyg,GG / buk,FR / jeg,NT / arj
1,BL / frm,TG / gyg,BG / gyg,FR / jeg,DE / frm
2,IN / frc,GS / dua,BG / gyg,MU / hlu,DE / frm
3,KS / frm,GS / dua,MA / buk,MU / hlu,
4,DE / frm,,MA / buk,,
5,,,,,
6,SH / bip,DE / frm,,RE / blb,
7,FR / jeg,DE / frm,,RE / blb,
8,EN / fsl,EN / fsl,,MA / buk,
9,SH / bip,EN / fsl,,MA / buk,


U21a


,0,1,2,3,4
0,KS / bip,NT / arj,FR / ebm,DE / scn,MA / ale
1,BL / bip,NT / arj,EN / dea,DE / scn,BG / gej
2,MA / ale,RE / blb,EN / dea,GS / dua,BG / gej
3,IN / frc,RE / blb,MU / vom,GS / dua,
4,DE / scn,,MU / vom,,
5,,,,,
6,SH / bip,DE / scn,,MA / ale,
7,EN / dea,DE / scn,,MA / ale,
8,FR / ebm,TG / gej,,GG / bip,
9,SH / bip,TG / gej,,GG / bip,


In [16]:
# Die letzte Stunde (10) soll nicht verwendet werden. Zudem soll auch Lektion 9 so wenig wie möglich verwendet werden

for l in lectures:
    for d in days:
        model.Add(start_vars[l] != 10+d*11)
        if lecture_details[l]['hours'] == 2:
            model.Add(start_vars[l] != 9+d*11)

slot_9_used = 0
for d in days:
    for l in lectures:
        used = model.NewBoolVar('slot_9_used_day_'+str(d)+'_lecture_'+str(l))
        slot_9_used += used
        if lecture_details[l]['hours'] == 1:
            model.Add(start_vars[l] == 9+d*11).OnlyEnforceIf(used)
            model.Add(start_vars[l] != 9+d*11).OnlyEnforceIf(used.Not())
            #slot_9_used += (start_vars[l] == 9+d*11)
        if lecture_details[l]['hours'] == 2:
            #slot_9_used += (start_vars[l] == 8+d*11)
            model.Add(start_vars[l] == 8+d*11).OnlyEnforceIf(used)
            model.Add(start_vars[l] != 8+d*11).OnlyEnforceIf(used.Not())
            
# Schönes Beispiel für nicht lineare Bedinungen. Natürlich hätten wir dies widerum mit No-Overlap Constraints regeln können
# Hier habe ich mich für die OnlyEnforceIf-Formulierung entschieden, um die Flexibilität von OR-Tools zu unterstreichen.
    
model.Minimize(slot_9_used) # Zielfunktion
# Wichtig: stellen Sie sicher, dass sie verstehen, dass Slot 10 auf keinen Fall verwendet werden darf, Slot 9 grundsätzlich schon. 
# Slot 10 wird explizit verboten, Slot 9 wird nur "unattraktiv gemacht" indem wir seine Verwendung in der Zielfunktion bestrafen

status = solver.SolveWithSolutionCallback(model, cb)
visualize()

Solution 12, time = 0.25 s, objective = 6
Solution 13, time = 0.26 s, objective = 5
Solution 14, time = 0.26 s, objective = 4
Solution 15, time = 0.26 s, objective = 3
Solution 16, time = 0.27 s, objective = 2
Solution 17, time = 0.28 s, objective = 1
Solution 18, time = 0.50 s, objective = 0
U21b


,0,1,2,3,4
0,MU / hlu,SH / bip,DE / frm,IN / frc,MA / buk
1,MU / hlu,TG / gyg,EN / fsl,BG / gyg,MA / buk
2,DE / frm,TG / gyg,EN / fsl,BG / gyg,MA / buk
3,DE / frm,GS / dua,FR / jeg,GG / buk,MA / buk
4,FR / jeg,GS / dua,FR / jeg,GG / buk,MA / buk
5,,,,,
6,SH / bip,DE / frm,,KS / frm,EN / fsl
7,SH / bip,DE / frm,,NT / arj,RE / blb
8,,BL / frm,,NT / arj,RE / blb
9,,,,,


U21a


,0,1,2,3,4
0,MA / ale,SH / bip,KS / bip,BL / bip,EN / dea
1,DE / scn,RE / blb,GS / dua,DE / scn,MA / ale
2,DE / scn,RE / blb,GS / dua,DE / scn,MA / ale
3,GG / bip,TG / gej,MA / ale,EN / dea,BG / gej
4,GG / bip,TG / gej,MA / ale,EN / dea,BG / gej
5,,,,,
6,SH / bip,NT / arj,,DE / scn,FR / ebm
7,SH / bip,NT / arj,,FR / ebm,MU / vom
8,,IN / frc,,FR / ebm,MU / vom
9,,,,,


Wie bereits erwähnt, wurde die Bedingung, dass im 9ten Zeitslot keine Lektion stattfinden soll 
als weiche Constraint definiert. Beim Lösen ist im Log ersichtlich, dass zuerst eine Lösung
gefunden wird, welche dies nicht respektiert. Anschliessend verbessert sich die Lösung, bis keine 
Verletzungen mehr vorhanden sind.

In [17]:
# Das selbe Fach soll nicht mehrfach am selben Tag in der selben Klasse unterrichtet werden

for c in classes:
    for d in days:
        for su in subjects:
            num_occurences = 0
            for l in lectures_by_class[c]:
                if lecture_details[l]['subject'] == su:
                    on_this_day = model.NewBoolVar(str(l)+'on_day_'+str(d))
                    num_occurences += on_this_day
                    model.Add(d*11 <= start_vars[l]).OnlyEnforceIf(on_this_day)
                    model.Add((d+1)*11 >= end_vars[l]).OnlyEnforceIf(on_this_day)
                    for s in slots:
                        if s >= d*11 and s < (d+1)*11:
                            model.Add(start_vars[l] != s).OnlyEnforceIf(on_this_day.Not())
            model.Add(num_occurences <= 1)
            
status = solver.SolveWithSolutionCallback(model, cb)
visualize()   

Solution 19, time = 0.85 s, objective = 6
Solution 20, time = 0.88 s, objective = 4
Solution 21, time = 0.88 s, objective = 3
Solution 22, time = 0.89 s, objective = 2
Solution 23, time = 0.89 s, objective = 1
Solution 24, time = 0.93 s, objective = 0
U21b


,0,1,2,3,4
0,BL / frm,RE / blb,EN / fsl,IN / frc,DE / frm
1,DE / frm,RE / blb,DE / frm,MA / buk,BG / gyg
2,DE / frm,EN / fsl,DE / frm,MA / buk,BG / gyg
3,FR / jeg,EN / fsl,MA / buk,GS / dua,SH / bip
4,FR / jeg,KS / frm,MA / buk,GS / dua,SH / bip
5,,,,,
6,GG / buk,,,FR / jeg,MA / buk
7,GG / buk,MU / hlu,,TG / gyg,NT / arj
8,SH / bip,MU / hlu,,TG / gyg,NT / arj
9,,,,,


U21a


,0,1,2,3,4
0,FR / ebm,BL / bip,TG / gej,DE / scn,MA / ale
1,FR / ebm,DE / scn,TG / gej,RE / blb,MU / vom
2,DE / scn,DE / scn,EN / dea,RE / blb,MU / vom
3,DE / scn,NT / arj,MA / ale,GS / dua,SH / bip
4,,NT / arj,MA / ale,GS / dua,SH / bip
5,,,,,
6,GG / bip,KS / bip,,MA / ale,FR / ebm
7,GG / bip,BG / gej,,MA / ale,EN / dea
8,SH / bip,BG / gej,,IN / frc,EN / dea
9,,,,,


Dies ist ein Beispiel für eine Constraint, welche in einem MIP deutlich eleganter formuliert werden kann. CP Modelle sind in einigen Belangen einfacher, MIP dafür in anderen.

In [18]:
# Gewisse Fächer, z.B. Sport, sollen innerhalb von 3 Tagen maximal einmal unterrichtet werden, damit die Regeneration gewährleistet ist

gap = model.NewIntVar(0,100,'slots_between_sports_lecture')
gap2 = model.NewIntVar(0,100,'slots_between_sports_lecture_2')
model.Add(gap == start_vars[1]-start_vars[16])
model.AddAbsEquality(gap2, gap)
model.Add(gap >= 11*3)
# Die Sportlektionen wurden hier mithilfe ihrer ID definiert
            
status = solver.SolveWithSolutionCallback(model, cb)
visualize()   

Solution 25, time = 1.21 s, objective = 6
Solution 26, time = 1.22 s, objective = 5
Solution 27, time = 1.23 s, objective = 4
Solution 28, time = 1.23 s, objective = 3
Solution 29, time = 1.23 s, objective = 2
Solution 30, time = 1.23 s, objective = 1
Solution 31, time = 1.24 s, objective = 0
U21b


,0,1,2,3,4
0,SH / bip,KS / frm,MA / buk,EN / fsl,SH / bip
1,MU / hlu,RE / blb,MA / buk,GS / dua,SH / bip
2,MU / hlu,RE / blb,IN / frc,GS / dua,EN / fsl
3,GG / buk,DE / frm,FR / jeg,MA / buk,EN / fsl
4,GG / buk,DE / frm,FR / jeg,MA / buk,DE / frm
5,,,,,
6,BL / frm,NT / arj,,FR / jeg,TG / gyg
7,BG / gyg,NT / arj,,DE / frm,TG / gyg
8,BG / gyg,MA / buk,,DE / frm,
9,,,,,


U21a


,0,1,2,3,4
0,SH / bip,FR / ebm,BL / bip,FR / ebm,SH / bip
1,GG / bip,FR / ebm,MU / vom,GS / dua,SH / bip
2,GG / bip,EN / dea,MU / vom,GS / dua,TG / gej
3,BG / gej,EN / dea,DE / scn,RE / blb,TG / gej
4,BG / gej,KS / bip,DE / scn,RE / blb,MA / ale
5,,,,,
6,MA / ale,DE / scn,,MA / ale,DE / scn
7,MA / ale,NT / arj,,MA / ale,DE / scn
8,,NT / arj,,IN / frc,EN / dea
9,,,,,


In [19]:
# Lehrer "bip" möchte maximal 2 Tage pro Woche arbeiten
# hierzu führen wir für jeden Tag eine Hilfsvariable ein:

bip_arbeitet = {d: model.NewBoolVar('bip_arbeitet_d_'+str(d)) for d in days}

# Nun können wir diese Variable mit den x Variablen verknüpfen:
for d in days:
    day_sum = 0
    for l in lectures_by_teacher['bip']:
        on_this_day = model.NewBoolVar('lecture_l_'+str(l)+'_on_day_d'+str(d))
        day_sum += on_this_day
        for s in slots:
            if s >= d*11 and s < (d+1)*11:
                model.Add(start_vars[l] != s).OnlyEnforceIf(on_this_day.Not())
    model.Add(bip_arbeitet[d]*10 >= day_sum)
            
model.Add(sum(bip_arbeitet.values()) <= 2)

solution = solver.SolveWithSolutionCallback(model, cb)
visualize()
for d in days:
    if solver.Value(bip_arbeitet[d]) > 0.5:
        print('bip arbeitet an Tag %i'%d)

Solution 32, time = 1.55 s, objective = 6
Solution 33, time = 1.56 s, objective = 3
Solution 34, time = 1.56 s, objective = 2
Solution 35, time = 1.57 s, objective = 1
Solution 36, time = 1.58 s, objective = 0
U21b


,0,1,2,3,4
0,GS / dua,FR / jeg,DE / frm,EN / fsl,GG / buk
1,GS / dua,FR / jeg,TG / gyg,MU / hlu,GG / buk
2,IN / frc,MA / buk,TG / gyg,MU / hlu,SH / bip
3,DE / frm,EN / fsl,MA / buk,DE / frm,SH / bip
4,DE / frm,EN / fsl,MA / buk,DE / frm,FR / jeg
5,,,,,
6,SH / bip,NT / arj,,KS / frm,MA / buk
7,BG / gyg,NT / arj,,RE / blb,MA / buk
8,BG / gyg,BL / frm,,RE / blb,
9,,,,,


U21a


,0,1,2,3,4
0,MA / ale,IN / frc,EN / dea,BG / gej,MA / ale
1,GG / bip,FR / ebm,EN / dea,BG / gej,MA / ale
2,GG / bip,FR / ebm,DE / scn,EN / dea,SH / bip
3,RE / blb,NT / arj,DE / scn,TG / gej,SH / bip
4,RE / blb,NT / arj,FR / ebm,TG / gej,KS / bip
5,,,,,
6,SH / bip,MU / vom,,DE / scn,GS / dua
7,DE / scn,MU / vom,,MA / ale,GS / dua
8,DE / scn,,,MA / ale,BL / bip
9,,,,,


bip arbeitet an Tag 0
bip arbeitet an Tag 4
